In [1]:
import torch
import numpy as np

from torchvision import transforms, datasets
import torchvision
import fiftyone.zoo as foz
import fiftyone as fo
from data_service import DataAugmentation

import random

from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
# The directory containing the dataset to import
dataset_dir = "~/fiftyone/coco-2017/validation/"


# dataset = fo.Dataset.from_dir(
#     dataset_dir=dataset_dir,
#     dataset_type=fo.types.ImageSegmentationDirectory,
#     labels_path='labels.json',
# )

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train", "test", "validation"],
    label_types=["segmentations"],
    max_samples=10000,
)

Found annotations at '/home/sumbel/fiftyone/coco-2017/raw/instances_train2017.json'
25 images found; downloading the remaining 9975
 100% |████████████████| 9975/9975 [20.7m elapsed, 0s remaining, 8.0 images/s]      
Writing annotations for 10000 downloaded samples to '/home/sumbel/fiftyone/coco-2017/train/labels.json'
Found test info at '/home/sumbel/fiftyone/coco-2017/raw/image_info_test2017.json'
25 images found; downloading the remaining 9975
 100% |████████████████| 9975/9975 [22.4m elapsed, 0s remaining, 6.6 images/s]       
Writing annotations for 10000 downloaded samples to '/home/sumbel/fiftyone/coco-2017/test/labels.json'
Found annotations at '/home/sumbel/fiftyone/coco-2017/raw/instances_val2017.json'
Only found 5000 (<10000) samples matching your requirements
25 images found; downloading the remaining 4975
 100% |████████████████| 4975/4975 [12.0m elapsed, 0s remaining, 9.5 images/s]       
Writing annotations to '/home/sumbel/fiftyone/coco-2017/validation/labels.json'
Data

In [2]:
session = fo.launch_app(dataset)





coco_train = datasets.CocoDetection(root = data_path, annFile = labels_path)
  

Found annotations at '/home/sumbel/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |████████████████████| 25/25 [3.7s elapsed, 0s remaining, 10.0 images/s]      
Writing annotations for 25 downloaded samples to '/home/sumbel/fiftyone/coco-2017/train/labels.json'
Test split is unlabeled; ignoring classes requirement
 100% |██████|    8.7Mb/8.7Mb [2.5s elapsed, 0s remaining, 4.4Mb/s]      
Extracting test info to '/home/sumbel/fiftyone/coco-2017/raw/image_info_test2017.json'
 100% |████████████████████| 25/25 [4.2s elapsed, 0s remaining, 6.2 images/s]       
Writing annotations for 25 downloaded samples to '/home/sumbel/fiftyone/coco-2017/test/labels.json'
Found annotations at '/home/sumbel/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'train'
 100% |███████████████████| 25/25 [203.9ms elapsed, 0s remaining, 123.3 samples/s]    
Loading 'coco-2017' split 'test'
Dat

NameError: name 'data_path' is not defined

In [2]:
dataset_path = "~/fiftyone/coco-2017/validation/"
labels_path = "/home/sumbel/fiftyone/coco-2017/validation/labels.json"

In [3]:
convert_tensor = transforms.ToTensor()
convert_image = transforms.ToPILImage()

In [4]:
train_set = datasets.ImageFolder(root=dataset_path, transform=DataAugmentation())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=2, shuffle=True)


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda', index=0)

In [ ]:
class DenseCL(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.backbone = torch.nn.Sequential(*(list(torchvision.models.resnet50(pretrained=False).children())[:-1]))
        

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
x = train_set[0][0][0]

In [13]:
nn.Identity(x)

Identity()

In [18]:
model = torchvision.models.resnet50(pretrained=False)

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
print(newmodel)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [20]:
feat_q = torch.ones(2, 2, 2, 2)

In [3]:
feat_q.size()

torch.Size([2, 2, 2, 2])

In [4]:
feat_q.size(1)

2

In [21]:
feat_q = feat_q.view(2, 2, 4)
feat_k = feat_q.clone()

In [23]:
feat_k

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [22]:
feat_q

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [15]:
feat = torch.ones(2, 4, 4)

In [24]:
torch.einsum('nca,ncb->nab', feat_q, feat_k)

tensor([[[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]]])

In [26]:
for n in range(feat_q.size(0)):
    for a in range(feat_q.size(2)):
        for b in range(feat_k.size(2)):
            total = 0
            for c in range(feat_k.size(1)):
                total += feat_q[n][c][a] * feat_k[n][c][b]
            feat[n][a][b] = total

In [27]:
feat

tensor([[[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]]])

In [9]:
t1 = torch.ones(4)
t2 = torch.zeros(4, 5)

In [16]:
t1 = t1.view(4, 1)

In [17]:
t2

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [24]:
t3  = torch.concat((t1, t2), dim=1) * 2

In [36]:
t1.size(-1)

1

In [23]:
t1.shape

torch.Size([4, 1])

In [25]:
t3

tensor([[2., 0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0., 0.]])

In [28]:
t3.argmax(dim=0)

tensor([0, 0, 0, 0, 0, 0])